In [1]:
import os
import cv2
import time
import joblib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, accuracy_score, precision_score
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, GlobalAvgPool2D, Dropout, concatenate
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

tf.__version__

'2.7.0'

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# -------- TEST USER ----------- #

TEST_USER      = '004'

BASE_DIR       = '../Dataset/'

# Google Drive
# BASE_DIR       = '/content/drive/MyDrive/Research/Hand Gesture/GitHub/'

IMG_DIR        = 'BW-Spatial-Path-Images/'
LOG_DIR        = 'Logs/'

USERS          = ['001', '002', '003', '004', '005', '006', '007', '008', '009',
                  '010', '011', '012', '013', '014', '015', '016', '017', '018',
                  '019', '020', '021', '022', '023', '024', '025']

# ------------------------------- Only Dynalic Gestures ------------------------------ #
GESTURES       = ['j', 'z', 'bad', 'deaf', 'fine', 'good', 'goodbye', 'hello', 'hungry',
                  'me', 'no', 'please', 'sorry', 'thankyou', 'yes', 'you']

PLANES         = ['XY', 'YZ', 'ZX']

BATCH_SIZE     = 32
IMG_LEN        = 160
IMG_SIZE       = (IMG_LEN, IMG_LEN)

# ------------- FOR THE GREATER GOOD :) ------------- #
DATASET_LEN    = 4000
TRAIN_LEN      = 3840
TEST_LEN       = 160

EPOCHS         = 50
LEARNING_RATE  = 0.001
DECAY          = 0.0

CONFIG         = '_L_7_S_160x160_E_7'

XY_WEIGHTS     = np.array([0.91, 0.75, 0.61, 0.63, 0.51, 0.66, 0.81, 0.65, 0.65, 0.31,
                           0.66, 0.29, 0.34, 0.64, 0.64, 0.31])
YZ_WEIGHTS     = np.array([0.73, 0.71, 0.70, 0.79, 0.76, 0.38, 0.80, 0.61, 0.58, 0.73,
                           0.49, 0.26, 0.26, 0.52, 0.59, 0.54])
ZX_WEIGHTS     = np.array([0.33, 0.66, 0.51, 0.54, 0.37, 0.51, 0.71, 0.30, 0.75, 0.41,
                           0.40, 0.27, 0.24, 0.61, 0.36, 0.49])

In [4]:
def load_data(plane):
    X_train = np.zeros((TRAIN_LEN, IMG_LEN, IMG_LEN, 3), dtype='uint8')
    X_test = np.zeros((TEST_LEN, IMG_LEN, IMG_LEN, 3), dtype='uint8')
    y_train = np.zeros((TRAIN_LEN, 1), dtype='uint8')
    y_test = np.zeros((TEST_LEN, 1), dtype='uint8')
    
    train_count = 0
    test_count = 0
        
    for gesture in GESTURES:
        print('loading data for ' + gesture + ' gesture on the ' + plane + ' plane ... ', end='')
        path = os.path.join(BASE_DIR, IMG_DIR, plane, gesture)
        for filename in os.listdir(path):
            img = cv2.imread(os.path.join(path, filename))
            resized = cv2.resize(img, IMG_SIZE)
            if filename[1:4] != TEST_USER:
                X_train[train_count, :] = resized
                y_train[train_count, 0] = GESTURES.index(gesture)
                train_count = train_count + 1
            else:
                X_test[test_count, :] = resized
                y_test[test_count, 0] = GESTURES.index(gesture)
                test_count = test_count + 1
                
        print('√')
        
    return X_train, X_test, y_train, y_test

def load_and_save_data(plane):
    X = np.zeros((DATASET_LEN, IMG_LEN, IMG_LEN, 3))
    y = np.zeros((DATASET_LEN, 1))
    
    train_count = 0
    test_count  = TRAIN_LEN
        
    for gesture in GESTURES:
        print('loading data for ' + gesture + ' gesture on the ' + plane + ' plane ... ', end='')
        path = os.path.join(BASE_DIR, IMG_DIR, plane, gesture)
        for filename in os.listdir(path):
            img = cv2.imread(os.path.join(path, filename))
            resized = cv2.resize(img, IMG_SIZE)
            if filename[1:4] != TEST_USER:
                X[train_count, :] = resized
                y[train_count, 0] = GESTURES.index(gesture)
                train_count = train_count + 1
            else:
                X[test_count, :] = resized
                y[test_count, 0] = GESTURES.index(gesture)
                test_count = test_count + 1
                
        print('√')

    joblib.dump(X, BASE_DIR + 'X_BW_' + plane + str(IMG_SIZE) + '.joblib')
    joblib.dump(y, BASE_DIR + 'Y_BW_' + plane + str(IMG_SIZE) + '.joblib')

def load_data_from_joblib(plane):
    print('Loading data for ' + plane + ' plane ... ', end='')
    X = joblib.load(BASE_DIR + 'X_BW_' + plane + str(IMG_SIZE) + '.joblib')
    y = joblib.load(BASE_DIR + 'Y_BW_' + plane + str(IMG_SIZE) + '.joblib')
    test_user = int(TEST_USER)
    X_train = X[:TRAIN_LEN, :, :, :]
    y_train = y[:TRAIN_LEN, :]
    X_test = X[TRAIN_LEN:, :, :, :]
    y_test = y[TRAIN_LEN:, :]

    print('√')

    return X_train, X_test, y_train, y_test

In [5]:
X_train_xy, X_test_xy, y_train_xy, y_test_xy = load_data('XY')
X_train_yz, X_test_yz, y_train_yz, y_test_yz = load_data('YZ')
X_train_zx, X_test_zx, y_train_zx, y_test_zx = load_data('ZX')

# Save to Google  Drive
# load_and_save_data('XY')X_train_xy, y_train_xy = shuffle(X_train_xy, y_train_xy)
# load_and_save_data('YZ')
# load_and_save_data('ZX')

# Load from Google Drive
# X_train_xy, X_test_xy, y_train_xy, y_test_xy = load_data_from_joblib('XY')
# X_train_yz, X_test_yz, y_train_yz, y_test_yz = load_data_from_joblib('YZ')
# X_train_zx, X_test_zx, y_train_zx, y_test_zx = load_data_from_joblib('ZX')

loading data for j gesture on the XY plane ... √
loading data for z gesture on the XY plane ... √
loading data for bad gesture on the XY plane ... √
loading data for deaf gesture on the XY plane ... √
loading data for fine gesture on the XY plane ... √
loading data for good gesture on the XY plane ... √
loading data for goodbye gesture on the XY plane ... √
loading data for hello gesture on the XY plane ... √
loading data for hungry gesture on the XY plane ... √
loading data for me gesture on the XY plane ... √
loading data for no gesture on the XY plane ... √
loading data for please gesture on the XY plane ... √
loading data for sorry gesture on the XY plane ... √
loading data for thankyou gesture on the XY plane ... √
loading data for yes gesture on the XY plane ... √
loading data for you gesture on the XY plane ... √
loading data for j gesture on the YZ plane ... √
loading data for z gesture on the YZ plane ... √
loading data for bad gesture on the YZ plane ... √
loading data for de

In [6]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

2022-01-07 14:50:47.139070: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-07 14:50:47.144854: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-07 14:50:47.145273: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-07 14:50:47.146087: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [7]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model.trainable = False

In [8]:
global_average_layer = GlobalAvgPool2D()
prediction_layer = Dense(len(GESTURES))

In [9]:
def get_cnn_block():
    input = tf.keras.layers.Input(shape=IMG_SHAPE)
    # x = data_augmentation(input)
    x = preprocess_input(input)
    x = base_model(x, training=False)
    x = global_average_layer(x)

    return input, x


def get_stacked_model(n):
    inputs = []
    CNNs = []

    for i in range(n):
        input_i, CNN_i = get_cnn_block()
        inputs.append(input_i)
        CNNs.append(CNN_i)

    x = concatenate(CNNs, axis=-1)
    x = Dropout(0.4)(x)
    # x = Dense(100, activation="relu")(x)
    # x = Dropout(0.4)(x)
    # x = Dense(20, activation='selu')(x)
    # x = Dropout(0.5)(x)
    output = prediction_layer(x)
    model = Model(inputs, output)
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    loss = SparseCategoricalCrossentropy(from_logits=True)
    model.compile(loss=loss, optimizer=opt, metrics=["accuracy"])
    
    return model


In [10]:
# def get_model():
#     inputs = tf.keras.Input(shape=IMG_SHAPE)
#     #     x = data_augmentation(inputs)
#     x = preprocess_input(inputs)
#     x = base_model(x, training=False)
#     x = global_average_layer(x)
#     x = tf.keras.layers.Dropout(0.2)(x)
#     outputs = prediction_layer(x)
#     model = tf.keras.Model(inputs, outputs)
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE, decay=DECAY),
#         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#         metrics=["accuracy"],
#     )
#     return model


In [11]:
model = get_stacked_model(3)
X_train_xy, X_train_yz, X_train_zx, y_train_xy = shuffle(
    X_train_xy, X_train_yz, X_train_zx, y_train_xy
)

history = model.fit(
    [X_train_xy, X_train_yz, X_train_zx],
    y_train_xy,
    validation_data=([X_test_xy, X_test_yz, X_test_zx], y_test_xy),
    batch_size=8,
    epochs=EPOCHS,
)


Epoch 1/50


2022-01-07 14:50:54.359703: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201


  2/480 [..............................] - ETA: 38s - loss: 3.5587 - accuracy: 0.0625      

2022-01-07 14:50:56.000329: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 16.73MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-01-07 14:50:56.067301: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 923.19MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-01-07 14:50:56.090945: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 923.19MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-01-07 14:50:56.102112: W tensorflow/core/common_runtime/bfc_allocator.cc:275] 

480/480 [==============================] - 48s 87ms/step - loss: 1.7767 - accuracy: 0.4604 - val_loss: 1.9535 - val_accuracy: 0.3750
Epoch 2/50
480/480 [==============================] - 40s 84ms/step - loss: 0.6392 - accuracy: 0.8219 - val_loss: 1.9261 - val_accuracy: 0.4375
Epoch 3/50
480/480 [==============================] - 41s 85ms/step - loss: 0.4025 - accuracy: 0.8826 - val_loss: 1.9329 - val_accuracy: 0.4563
Epoch 4/50
480/480 [==============================] - 41s 85ms/step - loss: 0.2994 - accuracy: 0.9055 - val_loss: 2.0380 - val_accuracy: 0.4563
Epoch 5/50
480/480 [==============================] - 50s 103ms/step - loss: 0.2378 - accuracy: 0.9255 - val_loss: 2.0688 - val_accuracy: 0.4500
Epoch 6/50
480/480 [==============================] - 41s 84ms/step - loss: 0.2029 - accuracy: 0.9344 - val_loss: 2.1946 - val_accuracy: 0.4437
Epoch 7/50
480/480 [==============================] - 41s 85ms/step - loss: 0.1822 - accuracy: 0.9378 - val_loss: 2.1820 - val_accuracy: 0.4500
Ep

In [ ]:
# model_xy = get_model()
# X_train_xy, y_train_xy = shuffle(X_train_xy, y_train_xy)
# history_xy = model_xy.fit(X_train_xy, y_train_xy, validation_data=(X_test_xy, y_test_xy), epochs=EPOCHS)

In [ ]:
# # prob_xy = tf.keras.Sequential([model_xy, tf.keras.layers.Softmax()])
# # y_pred_xy = prob_xy.predict(X_test_xy)
# y_pred_xy = model_xy.predict(X_test_xy)
# y_pred = np.argmax(y_pred_xy, axis=1)
# print(classification_report(y_test_xy.ravel(), y_pred, zero_division=0))
# prc_xy = precision_score(y_test_xy.ravel(), y_pred, zero_division=0, average=None)
# tf.keras.backend.clear_session()

In [ ]:
# model_yz = get_model()
# X_train_yz, y_train_yz = shuffle(X_train_yz, y_train_yz)
# history_yz = model_yz.fit(X_train_yz, y_train_yz, validation_data=(X_test_yz, y_test_yz), epochs=EPOCHS)

In [ ]:
# # prob_yz = tf.keras.Sequential([model_yz, tf.keras.layers.Softmax()])
# # y_pred_yz = prob_yz.predict(X_test_yz)
# y_pred_yz = model_yz.predict(X_test_yz)
# y_pred = np.argmax(y_pred_yz, axis=1)
# print(classification_report(y_test_yz.ravel(), y_pred, zero_division=0))
# prc_yz = precision_score(y_test_yz.ravel(), y_pred, zero_division=0, average=None)
# tf.keras.backend.clear_session()

In [ ]:
# model_zx = get_model()
# X_train_zx, y_train_zx = shuffle(X_train_zx, y_train_zx)
# history_zx = model_zx.fit(X_train_zx, y_train_zx, validation_data=(X_test_zx, y_test_zx), epochs=EPOCHS)

In [ ]:
# # prob_zx = tf.keras.Sequential([model_zx, tf.keras.layers.Softmax()])
# # y_pred_zx = prob_zx.predict(X_test_zx)
# y_pred_zx = model_zx.predict(X_test_zx)
# y_pred = np.argmax(y_pred_zx, axis=1)
# print(classification_report(y_test_zx.ravel(), y_pred, zero_division=0))
# prc_zx = precision_score(y_test_zx.ravel(), y_pred, zero_division=0, average=None)
# tf.keras.backend.clear_session()

In [ ]:
# y_total = y_pred_xy + y_pred_yz + y_pred_zx
# y_pred = np.argmax(y_total, axis=1)
# report = classification_report(y_test_xy.ravel(), y_pred, zero_division=0)
# print(report)

In [ ]:
# config = '\n\nTEST_USER ' + TEST_USER + ' T: ' + str(int(time.time())) + '\n'
# underline = '=====================================\n'
# log_dir = os.path.join(BASE_DIR, LOG_DIR)
# if not os.path.exists(log_dir):
#     os.mkdir(log_dir)
# f = open(os.path.join(log_dir, 'logs_sptl_bw' + CONFIG + '.txt'), 'a')
# f.write(config)
# f.write(underline)
# f.write(report)
# f.close()

In [ ]:
# config = TEST_USER + ' :'
# log_dir = os.path.join(BASE_DIR, LOG_DIR)
# if not os.path.exists(log_dir):
#     os.mkdir(log_dir)
# f = open(os.path.join(log_dir, 'prc_sptl_bw_xy' + CONFIG + '.txt'), 'a')
# f.write(config)
# f.write(np.array2string(prc_xy, precision=2, max_line_width=100) + '\n')
# f.close()

In [ ]:
# config = TEST_USER + ' :'
# log_dir = os.path.join(BASE_DIR, LOG_DIR)
# if not os.path.exists(log_dir):
#     os.mkdir(log_dir)
# f = open(os.path.join(log_dir, 'prc_sptl_bw_yz' + CONFIG + '.txt'), 'a')
# f.write(config)
# f.write(np.array2string(prc_yz, precision=2, max_line_width=100) + '\n')
# f.close()

In [ ]:
# config = TEST_USER + ' :'
# log_dir = os.path.join(BASE_DIR, LOG_DIR)
# if not os.path.exists(log_dir):
#     os.mkdir(log_dir)
# f = open(os.path.join(log_dir, 'prc_sptl_bw_zx' + CONFIG + '.txt'), 'a')
# f.write(config)
# f.write(np.array2string(prc_zx, precision=2, max_line_width=100) + '\n')
# f.close()